In [1]:
#importing libraries
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import joblib
%matplotlib inline
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
#importing spacy
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [167]:
#loading the data set
df=pd.read_csv(r'D:\Purity\HW saver\Raw_Skills_Dataset.csv')
# dataset shape
print(df.shape)
#getting the first 5 rows
df.head()

(34116, 1)


,RAW DATA
0,What ifs
1,seniority
2,familiarity
3,functionalities
4,Lambdas


In [168]:
#checking for null values
df.isnull().sum()

RAW DATA    0
dtype: int64

In [169]:
#checking variable types
df.dtypes

RAW DATA    object
dtype: object

In [170]:
#creating length of data column
df['Length_of_data'] = df['RAW DATA'].apply(len)
df.head()

,RAW DATA,Length_of_data
0,What ifs,8
1,seniority,9
2,familiarity,11
3,functionalities,15
4,Lambdas,7


In [171]:
# cleaning text data
#converting text to all lowercase
df['clean_data']=df['RAW DATA'].apply(lambda x: x.lower())
#removing the characters
for c in ['i.e.', 'e.g.', '&', '#']:
    df['clean_data'] = df['clean_data'].apply(lambda x: re.sub(c, "", x))
#removing e.g
#df['clean_data'] = df['clean_data'].apply(lambda x: re.sub("e.g.", "", x))
#removing non alphabatic characters
df['clean_data'] = df['clean_data'].apply(lambda x: re.sub('[^A-Za-z]', ' ',x))
#creating length of clean data column
df['Length_of_clean_data'] = df['clean_data'].apply(len)
df.head()

,RAW DATA,Length_of_data,clean_data,Length_of_clean_data
0,What ifs,8,what ifs,8
1,seniority,9,seniority,9
2,familiarity,11,familiarity,11
3,functionalities,15,functionalities,15
4,Lambdas,7,lambdas,7


In [172]:
#removing punctuation marks and stop words from the data
from nltk.corpus import stopwords

def data_preprocessing(data):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
   #Tokenizing into words 
    tokenized_word=word_tokenize(data)
    # Check characters to see if they are in punctuation
    nopunc = [char for char in tokenized_word if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    nostop=[word for word in nopunc if word not in stopwords.words('english')]
    
    #lemmatization
    lem = WordNetLemmatizer()
    return [lem.lemmatize(word) for word in nostop]

In [173]:
df['clean_data'].apply(data_preprocessing)
df['Length_of_clean_data'] = df['clean_data'].apply(len)
df.head()

,RAW DATA,Length_of_data,clean_data,Length_of_clean_data
0,What ifs,8,what ifs,8
1,seniority,9,seniority,9
2,familiarity,11,familiarity,11
3,functionalities,15,functionalities,15
4,Lambdas,7,lambdas,7


In [174]:
print(df.head(67))

           RAW DATA  Length_of_data       clean_data  Length_of_clean_data
0          What ifs               8         what ifs                     8
1         seniority               9        seniority                     9
2       familiarity              11      familiarity                    11
3   functionalities              15  functionalities                    15
4           Lambdas               7          lambdas                     7
..              ...             ...              ...                   ...
62            MySQL               5            mysql                     5
63           Oracle               6           oracle                     6
64     NoSQL Stores              12     nosql stores                    12
65  (i.e. Big Query              15        big query                    11
66       Clickhouse              10       clickhouse                    10

[67 rows x 4 columns]


In [72]:
#installing libraries for building the model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

In [175]:
#transforming with count vectorizer
#vectorizer = TfidfVectorizer(stop_words='english')
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(df.clean_data)
X.shape

(34116, 6871)

In [163]:
#creating the model
true_k = 2
model = KMeans(n_clusters=true_k)
model.fit(X)

KMeans(n_clusters=2)

In [176]:
# getting the centroids
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [177]:
order_centroids

array([[5677, 1149, 2221, ..., 4540, 4541,    0],
       [1527, 2249, 1707, ..., 3470, 5618,  710]], dtype=int64)

In [178]:
# printing the centroids into which clusters they belong to
for i in range(true_k):
    print(f'Cluster {i}'),
    for ind in order_centroids[i, :10]:
        print('%s' % terms[ind])

Cluster 0
skills
communication
excellent
strong
written
verbal
solving
problem
interpersonal
analytical
Cluster 1
data
experience
development
software
sql
aws
javascript
azure
cloud
technical
